---
title: "Card fraud model training"
date: 2021-06-04
type: technical_note
draft: false
---

## Create experiment

In [1]:
def experiment_wrapper():

    import os
    import sys
    import uuid
    import random
        
    import tensorflow as tf
    
    from hops import model as hops_model
    from hops import hdfs
    import hsfs
    # Create a connection
    connection = hsfs.connection(engine='training')
    # Get the feature store handle for the project's feature store
    fs = connection.get_feature_store()
    # Get training dataset
    td_meta = fs.get_training_dataset("card_fraud_model", 1)

    input_dim = 9
    encoding_dim = 4
    BATCH_SIZE = 32
    EPOCHS = 5
    
    train_input = td_meta.tf_data(target_name=None, is_training=True)
    train_input_not_processed = train_input.tf_record_dataset()

    def custom_impl(example):
        feature_names = [td_feature.name for td_feature in td_meta.schema] 
        x = [tf.cast(example[feature_name], tf.float32) for feature_name in feature_names]
        return x,x

    train_input_custum_processed = train_input_not_processed.map(lambda value: custom_impl(value))\
        .shuffle(EPOCHS * BATCH_SIZE)\
        .repeat(EPOCHS * BATCH_SIZE)\
        .cache()\
        .batch(BATCH_SIZE, drop_remainder=True)\
        .prefetch(tf.data.experimental.AUTOTUNE)


    autoencoder = tf.keras.Sequential()
    autoencoder.add(tf.keras.layers.Dense(16,  activation='selu', input_shape=(input_dim,)))
    autoencoder.add(tf.keras.layers.Dense(8,  activation='selu'))
    autoencoder.add(tf.keras.layers.Dense(4,    activation='linear', name="bottleneck"))
    autoencoder.add(tf.keras.layers.Dense(8,  activation='selu'))
    autoencoder.add(tf.keras.layers.Dense(16,  activation='selu'))
    autoencoder.add(tf.keras.layers.Dense(input_dim,  activation='selu'))

    
    # Compile the model.
    #autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    autoencoder.compile(loss=tf.keras.losses.MeanSquaredError(),
                        optimizer= tf.keras.optimizers.Adam(0.001),
                        metrics=tf.keras.metrics.MeanSquaredError()
                       )

    history = autoencoder.fit(
        train_input_custum_processed, 
        verbose=0,
        epochs=EPOCHS, 
        steps_per_epoch=1,
        validation_data=train_input_custum_processed,
        validation_steps=1
    )  
    
    metrics = {'loss': history.history['loss'][0]}

    # Export model
    # WARNING(break-tutorial-inline-code): The following code snippet is
    # in-lined in tutorials, please update tutorial documents accordingly
    # whenever code changes.

    export_path = os.getcwd() + '/model-' + str(uuid.uuid4())
    print('Exporting trained model to: {}'.format(export_path))
    
    tf.saved_model.save(autoencoder, export_path)

    print('Done exporting!')
        
    hops_model.export(export_path, "cardfraudmodel", metrics=metrics)    

    return metrics

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
16,application_1623086031838_0024,pyspark,idle,Link,Link


SparkSession available as 'spark'.


## Launch experiment

In [2]:
from hops import experiment
from hops import hdfs

experiment.launch(experiment_wrapper, name='card fraud model', local_logdir=True, metric_key='loss')

Finished Experiment 

('hdfs://rpc.namenode.service.consul:8020/Projects/cc_fraud_detection/Experiments/application_1623086031838_0024_1', {'loss': 2.97389817237854, 'log': 'Experiments/application_1623086031838_0024_1/output.log'})